# **ASSIGNMENT 2 CODE**
- Aim: Find predictors of performance models for these following target variables
    - Kda
    - Gold_earned
    - Damage_total
- The following are used as potential predictors:
    - Assists
    - Vision Score
    - damage_objectives
    - damage_turrets
    - damage_buildings
    - kills
    - deaths
    - Assists
    - time_cc
    - damage_taken
    - turret_kills
    

In [9]:
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import math
import sklearn
import seaborn as sns
import numpy.ma as ma
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits import mplot3d
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer
from IPython.display import Image
from random import randrange
pd.options.mode.chained_assignment = None  # default='warn'
# Insert the desired path here
import sys

In [8]:
%store -r path

# **STEP 1**
- Create the dataframe that you need
- Assume you need all of the data 

---



In [6]:
NA = pd.read_csv(f'{path}NAmatch.csv')
KR = pd.read_csv(f'{path}KRmatch.csv')
EU = pd.read_csv(f'{path}EUmatch.csv')

ALL = pd.concat([NA, KR, EU], ignore_index=True)

ALL_train, ALL_test = train_test_split(ALL, train_size=0.7)

# test kfolds cross validation
ALL = ALL.iloc[:, 4:18]

NameError: name 'path' is not defined

- Will only be using the numerical columns in the data set, so we need to filter these out

In [ ]:
ALL_train = ALL_train.iloc[:, 4:18]
ALL_test = ALL_test.iloc[:, 4:18]
# Need this when kfolds cross validation is done later
ALL_train_empty = ALL_train.copy()

# **STEP 2**
- Justify the imputation over removal of the data
- Determine the imputation method (mean, median)
- This will be done by examining the histogram lots

---




- First thing is to check how many entries per column that will be used in the data experiment are NaN values

In [ ]:
ALL_train.shape

(12059, 14)

In [ ]:
ALL_train.isna().sum()

assists              601
damage_objectives    580
damage_building      584
damage_turrets       582
deaths               581
kda                  596
kills                561
level                572
time_cc              613
damage_taken         604
turret_kills         604
vision_score         590
damage_total           0
gold_earned            0
dtype: int64

In [ ]:
ALL_test.shape

(5169, 14)

In [ ]:
ALL_test.isna().sum()

assists              280
damage_objectives    259
damage_building      250
damage_turrets       288
deaths               236
kda                  246
kills                267
level                252
time_cc              264
damage_taken         225
turret_kills         229
vision_score         265
damage_total           0
gold_earned            0
dtype: int64

- As seen in the following entries, we know that  there are ~600 missing entries for each column, but there are 12059 total entries for the training set
- For the test set, there are ~240 missing entries per column, but the total number of coluns are 5169
- Both of this amount to ~4% of the data. This is a small number and thus we can justify imputation of the mean without worry towards possibly skewing the data unintentionally

In [ ]:
def get_histograms(df):
    # Get the histogram from each column
    numbers = df.columns
    horizontal = (len(numbers)//2)
    col = 0
    # Make the figure to store subplots in
    fig, ax = plt.subplots(2, horizontal, figsize=(20, 10))
    plt.suptitle(f'Histograms of Every Numerical Variable in All Regions', fontsize=40, fontweight='bold')
    for i in range(2):
        for j in range(horizontal):
            # Get histogram of each numerical column in the
            # dataframe then save the file
            ax[i, j].hist(df.iloc[:,col], edgecolor='black', linewidth=1.2)
            if (col==0) or (col==7):
                ax[i, j].set_ylabel("Frequency", fontsize=20, fontdict=dict(weight='bold'))
            ax[i, j].set_xlabel(f'{numbers[col]}', 
                fontsize=10, fontdict=dict(weight='bold'))
            #plt.yticks(fontsize=35)
            #plt.xticks(fontsize=35)
            ax[i, j].grid(True)
            col = col + 1
    col = 4
    # set the spacing between subplots
    plt.subplots_adjust(wspace=0.5)
    plt.savefig(f'{path}histograms.png')
    plt.close()

In [ ]:
get_histograms(ALL_train)

In [ ]:
ALL_train.fillna(ALL_train.median(), inplace=True)

for col in ALL_test.columns:
    ALL_test[col].fillna(ALL_train.median(), inplace=True)

- Now you know that the histogram determines the best imputation value is the median, since all the values are skewed

# **STEP 3**
- Now we determine the features to select for each of the three targets:
    - kda
    - gold_earned
    - damage_total

---



## **KDA**
- Since you know the target variable is a continuous variable, we can try make a multivariate linear regression line to predit the data
- This means we should get the Pearson Coefficient values of each feature with KDA to decide which features we want to keep and discard

---



In [ ]:
# OUTLIER REMOVER FOR kda
def kda_outliers(df):
    new = df[df['kda'] < 30]
    return new

In [ ]:
ALL_train[[x for x in ALL_train.columns if x != 'kda']]

,assists,damage_objectives,damage_building,damage_turrets,deaths,kills,level,time_cc,damage_taken,turret_kills,vision_score,damage_total,gold_earned
2089,9.0,982.0,30624.0,982.0,4.0,7.0,14.0,23.0,24515.0,1.0,20.0,141810,10224
5051,1.0,1456.0,1456.0,1456.0,6.0,4.0,11.0,14.0,18969.0,0.0,7.0,58074,6861
2519,8.0,1043.0,17865.0,1518.0,6.0,9.0,14.0,16.0,28653.0,1.0,26.0,176947,13906
11888,5.0,84.0,84.0,84.0,7.0,0.0,13.0,0.0,14579.0,0.0,15.0,101156,8019
9495,7.0,2672.0,5425.0,2672.0,8.0,1.0,14.0,15.0,31215.0,1.0,20.0,123361,10341
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6919,14.0,1403.0,1462.0,1403.0,1.0,2.0,14.0,29.0,14914.0,0.0,99.0,24460,9263
9391,12.0,846.0,22564.0,846.0,6.0,11.0,17.0,8.0,28571.0,0.0,38.0,202401,14917
10206,11.0,788.0,788.0,788.0,5.0,12.0,14.0,25.0,17869.0,0.0,151.0,38894,16258
7292,23.0,750.0,1292.0,750.0,9.0,0.0,14.0,27.0,23224.0,0.0,103.0,20416,8005


In [ ]:
# HELPER FUNCTION TO CREATE THE PLOT FOR THE target
def pearson_plot(df, target, outlier_remove, folder, exclude=None):
    # Exclude is a list if we dont want any more variables
    if exclude:
        exclude.append(target)
        X_col = [x for x in df.columns if not x in exclude]
    else:
        X_col = [x for x in df.columns if x != target]

    return_dict = {'Features to Predict kda' : X_col, 'with_outliers': [], 'no_outliers': []}

    for col in X_col:
        # First create the big plot
        fig, ax = plt.subplots(1, 2, figsize=(20,10))
        plt.suptitle(f'Plot of {col} against {target}', fontsize=40, fontweight='bold')

        # With Outliers
        ax[0].set_xlabel(f'{col} with Outliers', fontsize=10, fontdict=dict(weight='bold'))
        
        X = df[col]
        y = df[target]

        ax[0].plot(X, y, 'o')

        m, b = np.polyfit(X, y, 1)

        ax[0].plot(X, m*X + b)

        # Append Pearson correlation of values with Outliers
        return_dict['with_outliers'].append(np.corrcoef(X, y)[0][1])

        # Wihtout Outliers
        ax[1].set_xlabel(f'{col} without Outliers', fontsize=10, fontdict=dict(weight='bold'))
        without = outlier_remove(df)
        
        X = without[col]
        y = without[target]

        ax[1].plot(X, y, 'o')

        m, b = np.polyfit(X, y, 1)

        ax[1].plot(X, m*X + b)

        #Add Additional Titles
        ax[0].set_ylabel(f'{target}', fontsize=20, fontdict=dict(weight='bold'))

        # Append Pearson correlation of values without Outliers
        return_dict['no_outliers'].append(np.corrcoef(X, y)[0][1])
    
        # Save the figure
        plt.savefig(f'{path}{folder}/{col}.png')
        plt.close()

    return return_dict



In [ ]:
return_dict = pearson_plot(ALL_train, 'kda', kda_outliers, 'kda_pearson')
pearson_kda = pd.DataFrame.from_dict(return_dict)

In [ ]:
pearson_kda

,Features to Predict kda,with_outliers,no_outliers
0,assists,0.307251,0.301740
1,damage_objectives,0.194858,0.197669
2,damage_building,0.269156,0.268499
3,damage_turrets,0.194204,0.197011
4,deaths,-0.589239,-0.591245
5,kills,0.323854,0.328270
6,level,0.134856,0.134487
7,time_cc,0.054426,0.052215
8,damage_taken,-0.132875,-0.134216
9,turret_kills,0.232992,0.235812


In [ ]:
check_collinearity_kda = ALL_train.drop('kda', axis=1).corr(method='pearson')
check_collinearity_kda[check_collinearity_kda > 0.5]

,assists,damage_objectives,damage_building,damage_turrets,deaths,kills,level,time_cc,damage_taken,turret_kills,vision_score,damage_total,gold_earned
assists,1.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5782,NaN,NaN
damage_objectives,NaN,1.000000,NaN,0.955524,NaN,NaN,NaN,NaN,NaN,0.762789,NaN,NaN,0.515487
damage_building,NaN,NaN,1.000000,NaN,NaN,0.506921,NaN,NaN,NaN,NaN,NaN,0.652847,0.574308
damage_turrets,NaN,0.955524,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.759931,NaN,NaN,0.516349
deaths,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
kills,NaN,NaN,0.506921,NaN,NaN,1.000000,0.524960,NaN,NaN,NaN,NaN,0.580848,0.759388
level,NaN,NaN,NaN,NaN,NaN,0.524960,1.000000,NaN,0.609598,NaN,NaN,0.747969,0.859589
time_cc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
damage_taken,NaN,NaN,NaN,NaN,NaN,NaN,0.609598,NaN,1.000000,NaN,NaN,0.540868,0.578462
turret_kills,NaN,0.762789,NaN,0.759931,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,0.527266


- We can see from the data that the correlation coefficients are all less than 0.5 EXCEPT DEATHS
- This means for the regression model to be created we are not expecting a strong model

## **GOLD_EARNED**

---

- Because this kind of data is discrete, it is not ideal to predict with a linear regression model
- Another option is to use a Poisson or Negative Binomial Regression for the data

In [ ]:
def gold_outliers(df):
    new = df[df['gold_earned'] < 25000]
    return new

In [ ]:
return_dict = pearson_plot(ALL_train, 'gold_earned', gold_outliers, 'gold_pearson')
pearson_gold = pd.DataFrame.from_dict(return_dict)

In [ ]:
pearson_gold

,Features to Predict kda,with_outliers,no_outliers
0,assists,0.198416,0.198408
1,damage_objectives,0.515487,0.513209
2,damage_building,0.574308,0.571966
3,damage_turrets,0.516349,0.514059
4,deaths,0.104034,0.103831
5,kda,0.190805,0.191021
6,kills,0.759388,0.758436
7,level,0.859589,0.860429
8,time_cc,0.063318,0.063614
9,damage_taken,0.578462,0.577933


In [ ]:
check_collinearity_gold_earned = ALL_train.drop('gold_earned', axis=1).corr(method='pearson')
check_collinearity_gold_earned[check_collinearity_gold_earned > 0.5]

,assists,damage_objectives,damage_building,damage_turrets,deaths,kda,kills,level,time_cc,damage_taken,turret_kills,vision_score,damage_total
assists,1.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5782,NaN
damage_objectives,NaN,1.000000,NaN,0.955524,NaN,NaN,NaN,NaN,NaN,NaN,0.762789,NaN,NaN
damage_building,NaN,NaN,1.000000,NaN,NaN,NaN,0.506921,NaN,NaN,NaN,NaN,NaN,0.652847
damage_turrets,NaN,0.955524,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.759931,NaN,NaN
deaths,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
kda,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
kills,NaN,NaN,0.506921,NaN,NaN,NaN,1.000000,0.524960,NaN,NaN,NaN,NaN,0.580848
level,NaN,NaN,NaN,NaN,NaN,NaN,0.524960,1.000000,NaN,0.609598,NaN,NaN,0.747969
time_cc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
damage_taken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.609598,NaN,1.000000,NaN,NaN,0.540868


## **DAMAGE_TOTAL**

---

- Because this kind of data is discrete, it is not ideal to predict with a linear regression model
- Another option is to use a Poisson or Negative Binomial Regression for the data

In [ ]:
def dmg_total_outliers(df):
    new = df[df['damage_total'] < 500000]
    return new

In [ ]:
return_dict = pearson_plot(ALL_train, 'damage_total', dmg_total_outliers, 'damage_total_pearson')
pearson_dmg_total = pd.DataFrame.from_dict(return_dict)

In [ ]:
pearson_dmg_total

,Features to Predict kda,with_outliers,no_outliers
0,assists,-0.073845,-0.075284
1,damage_objectives,0.452031,0.451152
2,damage_building,0.652847,0.652224
3,damage_turrets,0.451169,0.450746
4,deaths,0.053089,0.051748
5,kda,0.080911,0.081268
6,kills,0.580848,0.580847
7,level,0.747969,0.748607
8,time_cc,-0.089200,-0.089351
9,damage_taken,0.540868,0.541645


In [ ]:
check_collinearity_dmg_total = ALL_train.drop('damage_total', axis=1).corr(method='pearson')
check_collinearity_dmg_total[check_collinearity_dmg_total > 0.5]

,assists,damage_objectives,damage_building,damage_turrets,deaths,kda,kills,level,time_cc,damage_taken,turret_kills,vision_score,gold_earned
assists,1.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5782,NaN
damage_objectives,NaN,1.000000,NaN,0.955524,NaN,NaN,NaN,NaN,NaN,NaN,0.762789,NaN,0.515487
damage_building,NaN,NaN,1.000000,NaN,NaN,NaN,0.506921,NaN,NaN,NaN,NaN,NaN,0.574308
damage_turrets,NaN,0.955524,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.759931,NaN,0.516349
deaths,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
kda,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
kills,NaN,NaN,0.506921,NaN,NaN,NaN,1.000000,0.524960,NaN,NaN,NaN,NaN,0.759388
level,NaN,NaN,NaN,NaN,NaN,NaN,0.524960,1.000000,NaN,0.609598,NaN,NaN,0.859589
time_cc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
damage_taken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.609598,NaN,1.000000,NaN,NaN,0.578462


## **SOME NOTES**
- In all three of the target variables, there was no difference in excluding outliers from the dataset
- These outliers were defined to be target variable values that were a relatively far range from the rest of the values
- Key note should be to consider the correlation matrix formed by all 3 target variables. It is clear that many present multicollienarity

# **STEP 4**

---


- Now that we have some features that are selected, we can try make a model based on either all the features or some of the selected features
- The following models were used to assess the fit of the data:
    - Linear Regression Model
    - Polynomial Regression Model (Degree 2)
    - Polynomial Regression Model (Degree 3)
- The following methods were then used to assess the fit of the data:
    - Regression with All features
    - Regression with Filtered Features Based on Pearson Coefficient Value
    - K-Folds Cross Validation (k=10) for TRAINING Set
- Justification for Methods Are Found in the Paper

## **HELPER FUNCTIONS TO MAKE REGRESSION MODELS**
- All of these are to make the different regression models

---



In [ ]:
# Helper function to determine k-folds cross validation RMSE 
def kfolds_polyreg(df, target, k, degree):
    cv = KFold(n_splits=k)
    avg = 0
    for train_index, test_index in cv.split(df):
        # Impute train with median and impute test
        # with model
        train = df.iloc[train_index,:]
        test = df.iloc[test_index, :]

        # Fill both train and test with the median values of train
        train.fillna(train.median(), inplace=True)
        for col in test.columns:
            test[col].fillna(train[col].median(), inplace=True)

        # Split data into training and test sets    
        X_train, y_train = train.drop(target, axis=1), train[target]
        X_test, y_test = test.drop(target, axis=1), test[target]

        # Convert X_train and X_test so that it can be used for polynomial regression
        poly = PolynomialFeatures(degree=degree, include_bias=False)
        poly_train = poly.fit_transform(X_train)
        poly_test = poly.fit_transform(X_test)

        model = LinearRegression()
        model.fit(poly_train, y_train)
        y_pred = model.predict(poly_test)

        r2_val = r2_score(y_test, y_pred)
        avg = avg + r2_val
    
    return avg/k

In [ ]:
# Helper function to determine k-folds cross validation RMSE 
def kfolds_linreg(df, target, k):
    cv = KFold(n_splits=k)
    avg = 0
    for train_index, test_index in cv.split(df):
        # Impute train with median and impute test
        # with model
        train = df.iloc[train_index,:]
        test = df.iloc[test_index, :]

        # Fill both train and test with the median values of train
        train.fillna(train.median(), inplace=True)
        for col in test.columns:
            test[col].fillna(train[col].median(), inplace=True)

        # Split data into training and test sets    
        X_train, y_train = train.drop(target, axis=1), train[target]
        X_test, y_test = test.drop(target, axis=1), test[target]

        model = LinearRegression()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        r2_val = r2_score(y_test, y_pred)
        avg = avg + r2_val
    
    return avg/k

In [ ]:
# Create the final model using the train/test split
def create_linreg(train, test, target, name):
    # First impute training median into test set
    for col in test.columns:
        test[col].fillna(train[col].median(), inplace=True)
    
    # Factor out the variables first in train set
    X_train = train.drop(target, axis=1)
    y_train = train[target]

    X_test = test.drop(target, axis=1)
    y_test = test[target]

    # Now make the model of the regression line
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Get the residuals and plot the data
    residuals = np.subtract(y_test, y_pred)

    r2_val = r2_score(y_test, y_pred)

    # y_pred and residuals plot
    fig, ax = plt.subplots(figsize=(20,10))
    plt.suptitle(f'{target} LinReg Predicted Values and Residuals (R^2 = {r2_val:.5f})', fontsize=30, fontweight='bold')


    ax.scatter(y_pred, residuals)
    ax.set_xlabel(f'{target} Predicted Values', fontsize=20, fontdict=dict(weight='bold'))
    ax.set_ylabel(f'{target} Residuals', fontsize=20, fontdict=dict(weight='bold'))
    ax.axhline(y = 0, color = 'b')

    plt.savefig(f'{path}linreg/{name}_linreg_residual')
    plt.close()

    # Make another plot for y_pred and y_test plot
    fig, ax = plt.subplots(figsize=(20,10))
    plt.suptitle(f'{target} LinReg Predicted Values and Test Values (R^2 = {r2_val:.5f})', fontsize=30, fontweight='bold')


    ax.scatter(y_pred, y_test)
    ax.set_xlabel(f'{target} Predicted Values', fontsize=20, fontdict=dict(weight='bold'))
    ax.set_ylabel(f'{target} Test Values', fontsize=20, fontdict=dict(weight='bold'))

    plt.savefig(f'{path}linreg/{name}_linreg_predvstest')
    plt.close()

    return r2_val


In [ ]:
#  Helper function to get polynomial regression model
def create_polyreg(train, test, target, name, degree):
    # First impute training median into test set
    for col in test.columns:
        test[col].fillna(train[col].median(), inplace=True)
    
    # Split into features and target and train and test
    X_train, y_train = train.drop(target, axis=1), train[target]
    X_test, y_test = test.drop(target, axis=1), test[target]

    # Convert X_train and X_test so that it can be used for polynomial regression
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    poly_train = poly.fit_transform(X_train)
    poly_test = poly.fit_transform(X_test)

    # Fit the model into the training data
    model = LinearRegression()
    model.fit(poly_train, y_train)
    y_pred = model.predict(poly_test)

    # Get residuals and r2_val r2_score
    residuals = np.subtract(y_test, y_pred)
    r2_val = r2_score(y_test, y_pred)

    # y_pred and residuals plot
    fig, ax = plt.subplots(figsize=(20,10))
    plt.suptitle(f'{target} Poly{degree} Predicted Values and Residuals (R^2 = {r2_val:.5f})', fontsize=30, fontweight='bold')


    ax.scatter(y_pred, residuals)
    ax.set_xlabel(f'{target} Predicted Values', fontsize=20, fontdict=dict(weight='bold'))
    ax.set_ylabel(f'{target} Residuals', fontsize=20, fontdict=dict(weight='bold'))
    ax.axhline(y = 0, color = 'b')

    plt.savefig(f'{path}polyreg_{degree}/{name}_residual')
    plt.close()

    # Make another plot for y_pred and y_test plot
    fig, ax = plt.subplots(figsize=(20,10))
    plt.suptitle(f'{target} Poly{degree} Predicted Values and Test Values (R^2 = {r2_val:.5f})', fontsize=30, fontweight='bold')


    ax.scatter(y_pred, y_test)
    ax.set_xlabel(f'{target} Predicted Values', fontsize=20, fontdict=dict(weight='bold'))
    ax.set_ylabel(f'{target} Test Values', fontsize=20, fontdict=dict(weight='bold'))

    plt.savefig(f'{path}polyreg_{degree}/{name}_predvstest')
    plt.close()

    return r2_val



---






## **DICTIONARY TO TURN INTO CSV**
- This will be the summary statistics of the R^2 value of the different target values with the data
- Idea is to determine which target is most predictive of performance among the three target variables
- Whatever is the most predictive is the most indicative of performance


---



In [ ]:
r2_dict = {
    'Targets (Uses R^2)' : ['kda', 'gold_earned', 'damage_total'], 
    'LinearRegression': [], 
    'LinearRegressionFiltered': [], 
    'LinearRegressionKFolds': [],
    'PolynomialReg2' : [],
    'PolyReg2Filtered': [],
    'PolynomialReg2KFolds': [],
    'PolynomialReg3': [],
    'PolyReg3Filtered': [],
    'PolynomialReg3KFolds': [],
}



---



# **KDA**
---



## **FILTERED DATASET**


In [ ]:
pearson_kda[abs(pearson_kda['with_outliers']) < 0.1]

,Features to Predict kda,with_outliers,no_outliers
7,time_cc,0.054426,0.052215
10,vision_score,0.082812,0.081077
11,damage_total,0.080911,0.081208


In [ ]:
# Check pearson_kda again to find these values
filter_out = list(pearson_kda[abs(pearson_kda['with_outliers']) < 0.1]['Features to Predict kda'])
ALL_train_filtered = ALL_train.drop(filter_out, axis=1)
ALL_test_filtered = ALL_test.drop(filter_out, axis=1)



---



## **LINEAR REGRESSION MODEL WITH ALL FEATURES**

In [ ]:
linreg_r2 = create_linreg(ALL_train, ALL_test, 'kda', 'kda_all')
print(f'Linear Regression R^2 with All Features = {linreg_r2}')
r2_dict["LinearRegression"].append(linreg_r2)

Linear Regression R^2 with All Features = 0.5493224948612544




---



## **LINEAR REGRESSION MODEL WITH REMOVED FEATURES**

In [ ]:
linreg_r2_filtered = create_linreg(ALL_train_filtered, ALL_test_filtered, 'kda', 'kda_filtered')
print(f'Linear Regression R^2 with Filtered Features = {linreg_r2_filtered}')
r2_dict["LinearRegressionFiltered"].append(linreg_r2_filtered)

Linear Regression R^2 with Filtered Features = 0.5494418463874216




---



## **LINEAR REGRESSION EVALUATION USING K_FOLDS**


In [ ]:
# This is for the k-folds cross validation of the LinearRegression model
linreg_kfolds_r2 = kfolds_linreg(ALL_train_empty, 'kda', 10)
print(f'Linear Regression R^2 with kfolds with k=10: {linreg_kfolds_r2}')
r2_dict['LinearRegressionKFolds'].append(linreg_kfolds_r2)

Linear Regression R^2 with kfolds with k=10: 0.5466780761745211




---



## **POLYNOMIAL REGRESSION MODEL WITH ALL FEATURES**

In [ ]:
poly2_r2 = create_polyreg(ALL_train, ALL_test, 'kda', 'kda_all', 2)
print(f'Polynomial Regression (Degree = 2) R^2 with All Features = {poly2_r2}')
r2_dict['PolynomialReg2'].append(poly2_r2)

Polynomial Regression (Degree = 2) R^2 with All Features = 0.7645116823793795


In [ ]:
poly3_r2 = create_polyreg(ALL_train, ALL_test, 'kda', 'kda_all', 3)
print(f'Polynomial Regression (Degree = 3) R^2 with All Features = {poly3_r2}')
r2_dict['PolynomialReg3'].append(poly3_r2)

Polynomial Regression (Degree = 3) R^2 with All Features = -0.8322910072365077




---



## **POLYNOMIAL REGRESSION MODEL WITH FILTERED FEATURES**

In [ ]:
poly2_r2_filtered = create_polyreg(ALL_train_filtered, ALL_test_filtered, 'kda', 'kda_filtered', 2)
print(f'Polynomial Regress (Degree=2) R^2 with Filtered Features = {poly2_r2_filtered}')
r2_dict['PolyReg2Filtered'].append(poly2_r2_filtered)

Polynomial Regress (Degree=2) R^2 with Filtered Features = 0.7643110169555688


In [ ]:
poly3_r2_filtered = create_polyreg(ALL_train_filtered, ALL_test_filtered, 'kda', 'kda_filtered', 3)
print(f'Polynomial Regress (Degree=2) R^2 with Filtered Features = {poly3_r2_filtered}')
r2_dict['PolyReg3Filtered'].append(poly2_r2_filtered)

Polynomial Regress (Degree=2) R^2 with Filtered Features = 0.8297828813205609




---



## **POLYNOMIAL REGRESSION WITH KFOLDS CROSS VALIDATION**

In [ ]:
# This is for the k-folds cross validation of the LinearRegression model
poly2_kfolds_r2 = kfolds_polyreg(ALL_train_empty, 'kda', 10, 2)
print(f'Linear Regression R^2 with kfolds with k=10: {poly2_kfolds_r2}')
r2_dict['PolynomialReg2KFolds'].append(poly2_kfolds_r2)

Linear Regression R^2 with kfolds with k=10: 0.7528806222048553


In [ ]:
poly3_kfolds_r2 = kfolds_polyreg(ALL_train_empty, 'kda', 10, 3)
print(f'Linear Regression R^2 with kfolds with k=10: {poly3_kfolds_r2}')
r2_dict['PolynomialReg3KFolds'].append(poly3_kfolds_r2)

Linear Regression R^2 with kfolds with k=10: -2.346471469483781




---



# **GOLD_EARNED**




---



## **FILTERED DATASET**

In [ ]:
pearson_gold[abs(pearson_gold['with_outliers']) < 0.1]

,Features to Predict kda,with_outliers,no_outliers
8,time_cc,0.063318,0.063614
11,vision_score,0.065866,0.064291


In [ ]:
# Check pearson_gold again to find these values
filter_out = list(pearson_gold[abs(pearson_gold['with_outliers']) < 0.1]['Features to Predict kda'])
ALL_train_filtered = ALL_train.drop(filter_out, axis=1)
ALL_test_filtered = ALL_test.drop(filter_out, axis=1)



---



## **LINEAR REGRESSION MODEL WITH ALL FEATURES**

In [ ]:
linreg_r2 = create_linreg(ALL_train, ALL_test, 'gold_earned', 'gold_earned_all')
print(f'Linear Regression R^2 with All Features = {linreg_r2}')
r2_dict["LinearRegression"].append(linreg_r2)

Linear Regression R^2 with All Features = 0.9425741358923053




---



## **LINEAR REGRESSION WITH FILTERED FEATURES**

In [ ]:
linreg_r2_filtered = create_linreg(ALL_train_filtered, ALL_test_filtered, 'gold_earned', 'gold_earned_filtered')
print(f'Linear Regression R^2 with Filtered Features = {linreg_r2_filtered}')
r2_dict["LinearRegressionFiltered"].append(linreg_r2_filtered)

Linear Regression R^2 with Filtered Features = 0.9353913233904744




---



## **LINEAR REGRESSION EVALUATION USING K_FOLDS**

In [ ]:
# This is for the k-folds cross validation of the LinearRegression model
linreg_kfolds_r2 = kfolds_linreg(ALL_train_empty, 'gold_earned', 10)
print(f'Linear Regression R^2 with kfolds with k=10: {linreg_kfolds_r2}')
r2_dict['LinearRegressionKFolds'].append(linreg_kfolds_r2)

Linear Regression R^2 with kfolds with k=10: 0.9431164104172621




---



## **POLYNOMIAL REGRESSION MODEL WITH ALL FEATURES**

In [ ]:
poly2_r2 = create_polyreg(ALL_train, ALL_test, 'gold_earned', 'gold_earned_all', 2)
print(f'Polynomial Regression (Degree = 2) R^2 with All Features = {poly2_r2}')
r2_dict['PolynomialReg2'].append(poly2_r2)

Polynomial Regression (Degree = 2) R^2 with All Features = 0.9471196922921751


In [ ]:
poly3_r2 = create_polyreg(ALL_train, ALL_test, 'gold_earned', 'gold_earned_all', 3)
print(f'Polynomial Regression (Degree = 2) R^2 with All Features = {poly3_r2}')
r2_dict['PolynomialReg3'].append(poly3_r2)

Polynomial Regression (Degree = 2) R^2 with All Features = 0.8740161568620762




---



## **POLYNOMIAL REGRESSION MODEL WITH FILTERED FEATURES**

In [ ]:
poly2_r2_filtered = create_polyreg(ALL_train_filtered, ALL_test_filtered, 'gold_earned', 'gold_earned_filtered', 2)
print(f'Polynomial Regress (Degree=2) R^2 with Filtered Features = {poly2_r2_filtered}')
r2_dict['PolyReg2Filtered'].append(poly2_r2_filtered)

Polynomial Regress (Degree=2) R^2 with Filtered Features = 0.942950539171351


In [ ]:
poly3_r2_filtered = create_polyreg(ALL_train_filtered, ALL_test_filtered, 'gold_earned', 'gold_earned_filtered', 3)
print(f'Polynomial Regress (Degree=2) R^2 with Filtered Features = {poly3_r2_filtered}')
r2_dict['PolyReg3Filtered'].append(poly3_r2_filtered)

Polynomial Regress (Degree=2) R^2 with Filtered Features = 0.877383868225546




---



## **POLYNOMIAL REGRESSION WITH KFOLDS CROSS VALIDATION**

In [ ]:
# This is for the k-folds cross validation of the LinearRegression model
poly2_kfolds_r2 = kfolds_polyreg(ALL_train_empty, 'gold_earned', 10, 2)
print(f'Linear Regression R^2 with kfolds with k=10: {poly2_kfolds_r2}')
r2_dict['PolynomialReg2KFolds'].append(poly2_kfolds_r2)

Linear Regression R^2 with kfolds with k=10: 0.9499155111852445


In [ ]:
poly3_kfolds_r2 = kfolds_polyreg(ALL_train_empty, 'gold_earned', 10, 3)
print(f'Linear Regression R^2 with kfolds with k=10: {poly3_kfolds_r2}')
r2_dict['PolynomialReg3KFolds'].append(poly3_kfolds_r2)

Linear Regression R^2 with kfolds with k=10: 0.7686663275788097




---



# **DAMAGE_TOTAL**



---



## **FILTERED DATASET**

In [ ]:
pearson_dmg_total[abs(pearson_dmg_total['with_outliers']) < 0.1]

,Features to Predict kda,with_outliers,no_outliers
0,assists,-0.073845,-0.075284
4,deaths,0.053089,0.051748
5,kda,0.080911,0.081268
8,time_cc,-0.089200,-0.089351


In [ ]:
# Check pearson_dmg_total again to find these values
filter_out = list(pearson_dmg_total[abs(pearson_dmg_total['with_outliers']) < 0.1]['Features to Predict kda'])
ALL_train_filtered = ALL_train.drop(filter_out, axis=1)
ALL_test_filtered = ALL_test.drop(filter_out, axis=1)



---



## **LINEAR REGRESSION MODEL WITH ALL FEATURES**

In [ ]:
linreg_r2 = create_linreg(ALL_train, ALL_test, 'damage_total', 'damage_total_all')
print(f'Linear Regression R^2 with All Features = {linreg_r2}')
r2_dict["LinearRegression"].append(linreg_r2)

Linear Regression R^2 with All Features = 0.8896762593107319




---



## **LINEAR REGRESSION MODEL WITH FILTERED FEATURES**

In [ ]:
linreg_r2_filtered = create_linreg(ALL_train_filtered, ALL_test_filtered, 'damage_total', 'damage_total_filtered')
print(f'Linear Regression R^2 with Filtered Features = {linreg_r2_filtered}')
r2_dict["LinearRegressionFiltered"].append(linreg_r2_filtered)

Linear Regression R^2 with Filtered Features = 0.8693080257587391




---



## **LINEAR REGRESSION EVALUATION WITH KFOLDS CROSS VALIDATION**

In [ ]:
# This is for the k-folds cross validation of the LinearRegression model
linreg_kfolds_r2 = kfolds_linreg(ALL_train_empty, 'damage_total', 10)
print(f'Linear Regression R^2 with kfolds with k=10: {linreg_kfolds_r2}')
r2_dict['LinearRegressionKFolds'].append(linreg_kfolds_r2)

Linear Regression R^2 with kfolds with k=10: 0.8810480354608279




---



## **POLYNOMIAL REGRESSION MODEL WITH ALL FEATURES**

In [ ]:
poly2_r2 = create_polyreg(ALL_train, ALL_test, 'damage_total', 'damage_total_all', 2)
print(f'Polynomial Regression (Degree = 2) R^2 with All Features = {poly2_r2}')
r2_dict['PolynomialReg2'].append(poly2_r2)

Polynomial Regression (Degree = 2) R^2 with All Features = 0.9070774288478524


In [ ]:
poly3_r2 = create_polyreg(ALL_train, ALL_test, 'damage_total', 'damage_total_all', 3)
print(f'Polynomial Regression (Degree = 2) R^2 with All Features = {poly3_r2}')
r2_dict['PolynomialReg3'].append(poly3_r2)

Polynomial Regression (Degree = 2) R^2 with All Features = 0.9053036723145104




---



## **POLYNOMIAL REGRESSION WITH FILTERED FEATURES**

In [ ]:
poly2_r2_filtered = create_polyreg(ALL_train_filtered, ALL_test_filtered, 'damage_total', 'damage_total_filtered', 2)
print(f'Polynomial Regress (Degree=2) R^2 with Filtered Features = {poly2_r2_filtered}')
r2_dict['PolyReg2Filtered'].append(poly2_r2_filtered)

Polynomial Regress (Degree=2) R^2 with Filtered Features = 0.8881209264497643


In [ ]:
poly3_r2_filtered = create_polyreg(ALL_train_filtered, ALL_test_filtered, 'damage_total', 'damage_total_filtered', 3)
print(f'Polynomial Regress (Degree=2) R^2 with Filtered Features = {poly3_r2_filtered}')
r2_dict['PolyReg3Filtered'].append(poly3_r2_filtered)

Polynomial Regress (Degree=2) R^2 with Filtered Features = 0.8894179182309134




---



## **POLYNOMIAL REGRESSION EVALUATION WITH KFOLDS CROSS VALIDATION**

In [ ]:
# This is for the k-folds cross validation of the LinearRegression model
poly2_kfolds_r2 = kfolds_polyreg(ALL_train_empty, 'damage_total', 10, 2)
print(f'Linear Regression R^2 with kfolds with k=10: {poly2_kfolds_r2}')
r2_dict['PolynomialReg2KFolds'].append(poly2_kfolds_r2)

Linear Regression R^2 with kfolds with k=10: 0.8959767758244958


In [ ]:
poly3_kfolds_r2 = kfolds_polyreg(ALL_train_empty, 'damage_total', 10, 3)
print(f'Linear Regression R^2 with kfolds with k=10: {poly3_kfolds_r2}')
r2_dict['PolynomialReg3KFolds'].append(poly3_kfolds_r2)

Linear Regression R^2 with kfolds with k=10: 0.8888651646935049




---



# **CONCLUSION OF DATA STATS**

In [ ]:
r2_frame = pd.DataFrame.from_dict(r2_dict)
r2_frame.T

,0,1,2
Targets (Uses R^2),kda,gold_earned,damage_total
LinearRegression,0.549322,0.942574,0.889676
LinearRegressionFiltered,0.549442,0.935391,0.869308
LinearRegressionKFolds,0.546678,0.943116,0.881048
PolynomialReg2,0.764512,0.94712,0.907077
PolyReg2Filtered,0.764311,0.942951,0.888121
PolynomialReg2KFolds,0.752881,0.949916,0.895977
PolynomialReg3,-0.832291,0.874016,0.905304
PolyReg3Filtered,0.764311,0.877384,0.889418
PolynomialReg3KFolds,-2.346471,0.768666,0.888865


- As it can be seen from the graphs, the best performing datset was the gold_earned dataset. This means gold_earned is hte best predictor for performance based on the other numerical variables present in the dataset

# **EXTRA FUNCTIONS**

---



In [ ]:
# Create the final model using the train/test split
def create_expreg(train, test, target, name):
    # The exponential regression function cannot take values less
    # than zero, so we need to remove those
    new_train = train[train['kda'] > 0]
    new_test = test[test['kda'] > 0]


    # First impute training median into test set
    for col in test.columns:
        test[col].fillna(train[col].median(), inplace=True)
    
    # Factor out the variables first in train set
    X_train = new_train.drop(target, axis=1)
    y_train = new_train[target]

    X_test = new_test.drop(target, axis=1)
    y_test = new_test[target]

    # Now make the model of the regression line
    model = LinearRegression()
    model.fit(X_train, np.log(y_train))
    y_pred = model.predict(X_test)

    # Get the residuals and plot the data
    residuals = np.subtract(y_test, y_pred)

    r2_val = r2_score(y_test, y_pred)

    # y_pred and residuals plot
    fig, ax = plt.subplots(figsize=(20,10))
    plt.suptitle(f'{target} Predicted Values and Residuals (R^2 = {r2_val:.5f})', fontsize=40, fontweight='bold')


    ax.scatter(y_pred, residuals)
    ax.set_xlabel(f'{target} Predicted Values', fontsize=20, fontdict=dict(weight='bold'))
    ax.set_ylabel(f'{target} Residuals', fontsize=20, fontdict=dict(weight='bold'))
    ax.axhline(y = 0, color = 'b')

    plt.savefig(f'{path}{name}_expreg_residual')
    plt.close()

    # Make another plot for y_pred and y_test plot
    fig, ax = plt.subplots(figsize=(20,10))
    plt.suptitle(f'{target} Predicted Values and Test Values (R^2 = {r2_val:.5f})', fontsize=40, fontweight='bold')


    ax.scatter(y_pred, y_test)
    ax.set_xlabel(f'{target} Predicted Values', fontsize=20, fontdict=dict(weight='bold'))
    ax.set_ylabel(f'{target} Test Values', fontsize=20, fontdict=dict(weight='bold'))

    plt.savefig(f'{path}{name}_expreg_predvstest')
    plt.close()

    return r2_val